Trying to replicate "Automatic Brain Tumor Segmentation by Subject Specific Modification of Atlas Priors" (Prastawa et al., 2003).

Create tumor prior
------------------

1. Co-register T1 and T1 contrast-enhanced.
1. Subtract T1 from T1 contrast-enhanced.
1. Use subtracted volume to make a probability mask.

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import division, print_function

import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

def plot(arr, title=None):
    """Plot a 2D slice."""
    plt.imshow(arr, cmap='gray', origin='lower')
    if title is not None:
        plt.title(title)
    plt.show()

def transform_0_1(arr):
    """Transform data to range [0, 1]."""
    return (arr - arr.min()) / (arr.max() - arr.min())

In [ ]:
data_dir = '/om/user/jakubk/meningioma/data'
subject = 'subj-005'

T1_contrast = '{}_gad-T1w.nii.gz'.format(subject)
T1_contrast_nii = nib.load(op.join(data_dir, subject, 'anat', T1_contrast))
T1_contrast = T1_contrast_nii.get_data()

T1 = '{}_T1w.nii.gz'.format(subject)
T1_nii = nib.load(op.join(data_dir, subject, 'anat', T1))
T1 = T1_nii.get_data()

T1_flirt = '{}_T1w-flirt.nii.gz'.format(subject)  # Registered to T1 contrast.
T1_flirt_nii = nib.load(op.join(data_dir, subject, 'anat', T1_flirt))
T1_flirt = T1_flirt_nii.get_data()

T2 = '{}_T2w.nii.gz'.format(subject)
T2_nii = nib.load(op.join(data_dir, subject, 'anat', T2))
T2 = T2_nii.get_data()

T2_flirt = '{}_T2w-flirt.nii.gz'.format(subject)  # Registered to T1 contrast.
T2_flirt_nii = nib.load(op.join(data_dir, subject, 'anat', T2_flirt))
T2_flirt = T2_flirt_nii.get_data()

In [ ]:
print("T1 contrast shape", T1_contrast.shape)
print("T1 shape", T1.shape)
print("T1 FLIRT shape", T1_flirt.shape)
print("T2 shape", T2.shape)
print("T2 FLIRT shape", T2_flirt.shape)

In [ ]:
slice_ = 135
plot(T1_contrast[:, :, slice_].T, "T1 contrast-enhanced")
plot(T1_flirt[:, :, slice_].T, "T1 registered to T1 CE")

In [ ]:
# diff = T1_post - T1_pre
diff = (T1_contrast[:, :, slice_] - T1_flirt[:, :, slice_]).T
plot(diff, "Difference image")

diff_0_1 = transform_0_1(T1_contrast[:, :, slice_]) - transform_0_1(T1_flirt[:, :, slice_])
plot(diff_0_1.T, "Difference image scaled")

In [ ]:
plt.hist(diff_0_1)
plt.show()

Why does it not look like the difference image in the paper by Prastawa and colleagues (2003)?

<img src='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2430604/bin/nihms53796f3.jpg', width=500>

*From left to right: T1 contrast-enhanced, T1, difference image (T1 CE - T1)*

In [ ]:
print("Difference image array\n----------------------")
print("Minimum:\t{:.2f}".format(diff.min()))
print("Max:\t\t{:.2f}".format(diff.max()))
print("Mean:\t\t{:.2f}".format(diff.mean()))
print("St. dev.:\t{:.2f}".format(diff.std()))